In [15]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [16]:
#constants, coefficients
a = 1
R = 3
F_0 = 5
m = 1
phi = np.arccos(a/R)
cycles = 10

coefficient_drag_water = 7/10
coefficient_drag_air = 1/10
mechanical_friction = 2/10

In [17]:
#function that calculates the theta state
def evolve_theta_set_drag(theta, theta_prime, theta_double_prime, time_step):
    theta_prime = theta_prime + theta_double_prime * time_step #speed of the angle

    theta = theta + theta_prime * time_step #the current angle
    if theta > 2*np.pi - phi:
      theta = -phi

    for i in range(cycles):
      if (theta > -phi + 2 * i * np.pi) and (theta < phi + 2 * i * np.pi):
        theta_double_prime = F_0/(m*R)*(np.cos(theta) - coefficient_drag_water/2*theta_prime) - mechanical_friction
        break
      else:
        theta_double_prime = - coefficient_drag_air/2*theta_prime - mechanical_friction

    return theta, theta_prime, theta_double_prime

In [18]:
#calculating initial state
theta = -phi #theta starts at -phi
theta_double_prime = F_0/(m*R)*np.cos(theta)
theta_prime = 0 #initially not moving in the water
time = 0
time_step = 0.005 #time increment
phi = np.arccos(a/R)

theta_set = [(time, theta, theta_prime, theta_double_prime)]

#while loop valid only when theta is less than phi. Use time
while time < 40:
    theta, theta_prime, theta_double_prime = evolve_theta_set_drag(theta, theta_prime, theta_double_prime, time_step)
    time = time + time_step
    theta_set.append((time, theta, theta_prime, theta_double_prime))

theta_set = pd.DataFrame(theta_set, columns = ['time', 'theta', 'theta_prime', 'theta_double_prime'])

In [19]:
fig_len= 700
fig = go.Figure()
fig.update_layout(title = 'Theta State Plot', xaxis_title = 'time', yaxis_title = 'theta set',
                  height = fig_len, width = fig_len + 200
                 )
fig.add_trace(go.Scatter(x=theta_set['time'], y = theta_set['theta'],
                    mode='lines',
                    name='theta'))
fig.add_trace(go.Scatter(x=theta_set['time'], y = theta_set['theta_prime'],
                    mode='lines',
                    name='theta prime'))
fig.add_trace(go.Scatter(x=theta_set['time'], y = theta_set['theta_double_prime'],
                    mode='lines',
                    name='theta double prime'))

fig.add_hline(y = -phi, line_dash = 'dash', line_color = 'blue',
                annotation_text = "-Phi", annotation_position="bottom right")
fig.add_hline(y = phi, line_dash = 'dash', line_color = 'blue',
                annotation_text = "+Phi", annotation_position="bottom right")


#initial acceleration
fig.add_hline(y = F_0 /(m*R) * np.cos(phi), line_dash = 'dash', line_color = 'green')

fig.add_vline(x = 2.75, line_dash = 'dash', line_color = 'green')
fig.show()